<h2><h2 align = "middle"><b>United States Airline Company</b></h2>

## **Introduction** 
A United States airline company aims to enter the domestic market and has decided to start with 5 round trip routes between medium and large airports. In this challenge, three datasets have been used which provide information about flights, tickets and airports for the 1st quarter of the year 2019.   
<br>


**Business Objectives**
1. Using flights data consisting of competitor's flight information, find the most profitable routes which could be suggested to the airline company to build their business. 
2. Identifying key performance indicators for round trip routes that would help the company track their success in the future. Provide recommendations and statistical reasoning behind chosing the KPIs for the best routes.  
<br>
<br>

**Data Used**
1. Flights: Provides information about different operating carriers throughout the quarter. Consists of ~1.9 million rows and 16 feature variables.
2. Tickets: Provides passenger information travelling from one place to another. Consists of ~1.1 million rows and 12 feature variables. This is a sample data. 
3. Airport Codes: Provides airport information. Consists of ~55000 rows and 8 columns. The dataset has 52426 unique airport information.
<br>
<br>

**Assumptions**
1. Each airplane is dedicated to one round trip route between the 2 airports for Tickets dataset.
2. Cost Calcuation for total round trip considers Distance from Origin-> Destination and Destination -> Origin. Hence, Multiplied by 2.
3. Quarterly occupancy rates are same throughout the quarter.


## **Import data and libraries**

In [1]:
# Unzipping and load data from GitHub repository
%%sh
wget -qq https://github.com/CapitalOneRecruiting/DA-Airline-Data-Challenge/blob/main/data.zip?raw=true
unzip "/content/data.zip?raw=true"

Archive:  /content/data.zip?raw=true
  inflating: Airline_Challenge_Metadata.xlsx  
  inflating: __MACOSX/._Airline_Challenge_Metadata.xlsx  
  inflating: Airport_Codes.csv       
  inflating: __MACOSX/._Airport_Codes.csv  
  inflating: Flights.csv             
  inflating: __MACOSX/._Flights.csv  
  inflating: Tickets.csv             
  inflating: __MACOSX/._Tickets.csv  


In [2]:
# Importing libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from collections import Counter
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# plotly.offline.init_notebook_mode(connected=True)

## **Reading Data Files**

In [3]:
flights = pd.read_csv('Flights.csv', low_memory=False)
ac = pd.read_csv('Airport_Codes.csv')
tickets = pd.read_csv("Tickets.csv")

## **Data Pre-Processing**

**Identifying nulls:**

In [4]:
# Checking for na values in the Flights Dataset
na_flights = pd.DataFrame(flights.isna().sum())
na_flights.insert(1,"% na values",na_flights.values*100/flights.shape[0])  
na_flights.transpose()

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DESTINATION,DEST_CITY_NAME,DEP_DELAY,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE
0,0.0,0.0,12156.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50351.000000,55991.00000,0.0,56551.000000,630.000000,310.000000
% na values,0.0,0.0,0.634485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.628079,2.92246,0.0,2.951689,0.032883,0.016181


Since, the na values are very insignificant in amount (less than 3% of whole dataset), they can be dropped.

In [5]:
flights = flights.dropna()
na_flights = pd.DataFrame(flights.isna().sum())
na_flights.insert(1,"% na values",na_flights.values*100/flights.shape[0])  
na_flights.transpose()

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DESTINATION,DEST_CITY_NAME,DEP_DELAY,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
% na values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Checking for na values in the Tickets Dataset and dropping them since they are very insginificant compared to the whole dataset
na_tckt = pd.DataFrame(tickets.isna().sum())
na_tckt.insert(1,"% na values",na_tckt.values*100/tickets.shape[0])  
na_tckt.transpose()

tickets = tickets.dropna()

In [7]:
# Checking for na values in the Airport_Codes Dataset.
na_ac = pd.DataFrame(ac.isna().sum())
na_ac.insert(1,"% na values",na_ac.values*100/ac.shape[0])  
na_ac

# ac = ac.dropna()

,0,% na values
TYPE,0,0.000000
NAME,0,0.000000
ELEVATION_FT,7015,12.669544
CONTINENT,27843,50.286261
ISO_COUNTRY,247,0.446098
MUNICIPALITY,5706,10.305406
IATA_CODE,46187,83.416713
COORDINATES,0,0.000000


**Handling NaN Values and Special Characters:**

* FL_DATE column contains date in different format, which needs to be transformed into a consistent format.
* DISTANCE column has three abnormal values, out of which '****' is more than 2000 times. Some filler is required to eliminate those values; so they shall be considered as NaN's. Also, replacing negative values with nan so that distance can be backfilled from other origin-destination pair.
* ITIN_FARE column Replacing '$' with empty space.

Assumptions considered for analysis further:
- Flights Dataset: Only non-cancelled flights have been considered
- Tickets Dataset: Roundtrip = 1 is considered


In [8]:
## Flights Dataset

## Converting date into consistent date-time format
flights['FL_DATE'] = pd.to_datetime(flights.FL_DATE).dt.strftime('%m/%d/%Y')

## Handling Special Characters in column DISTANCE
flights['DISTANCE'] = flights['DISTANCE'].astype(str).replace({"Hundred":np.nan,"Twenty":np.nan,"****":np.nan,"-1947.0":np.nan,"-198.0":np.nan,-1947:np.nan,-198:np.nan}).astype(float)
flights = flights.sort_values(['ORIGIN',"DESTINATION"])
flights['DISTANCE'] = flights['DISTANCE'].fillna(method='ffill')  # forward-filling rows with na

# Considering only non-cancelled flights
flights = flights[flights.CANCELLED == 0.0]

## Tickets Dataset
tickets['ITIN_FARE'] = tickets['ITIN_FARE'].str.replace('$', '').astype(float)    # 
tickets = tickets[tickets["ROUNDTRIP"]==1]

## **Exploratory Data Analysis**

**Flights Dataset**

In [9]:
flights.shape

(1858595, 16)

In [10]:
flights.describe()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,ARR_DELAY,CANCELLED,DISTANCE,OCCUPANCY_RATE
count,1.858595e+06,1.858595e+06,1.858595e+06,1.858595e+06,1858595.0,1.858595e+06,1.858595e+06
mean,1.268589e+04,1.268599e+04,1.072842e+01,5.651617e+00,0.0,7.724272e+02,6.502243e-01
std,1.522151e+03,1.521664e+03,4.997027e+01,5.242202e+01,0.0,5.822027e+02,2.020090e-01
min,1.013500e+04,1.013500e+04,-6.300000e+01,-9.400000e+01,0.0,-1.947000e+03,3.000000e-01
25%,1.129200e+04,1.129200e+04,-6.000000e+00,-1.500000e+01,0.0,3.440000e+02,4.800000e-01
50%,1.288900e+04,1.288900e+04,-2.000000e+00,-6.000000e+00,0.0,6.120000e+02,6.500000e-01
75%,1.405700e+04,1.405700e+04,7.000000e+00,8.000000e+00,0.0,1.013000e+03,8.300000e-01
max,1.621800e+04,1.621800e+04,2.941000e+03,2.923000e+03,0.0,9.898000e+03,1.000000e+00


In [ ]:
flights.median()

ORIGIN_AIRPORT_ID    12889.00
DEST_AIRPORT_ID      12889.00
DEP_DELAY               -2.00
ARR_DELAY               -6.00
CANCELLED                0.00
DISTANCE               612.00
OCCUPANCY_RATE           0.65
dtype: float64

The Flights dataset initially had about 1.9 million datapoints. However, after data pre-processing, it now contains about 1.85 million datapoints with 16 feature variables. 

Skewness: ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID, DEP_DELAY, ARR_DELAY, DISTANCE are all positively skewed since mean > median. Canceled and occupancy rates are almmost normally distributed. However, looking at the max, min values the dataset might have many outliers.


**Delays by Day-Of-Week**

In [ ]:
# Delay by weekday. Since I'm not checking delays based on location, I can create a new column by summing delays at departure and arrival.
flt = flights  # using a duplicate dataframe as the extra column would not be required later
flt['Total_Delay'] = flt['ARR_DELAY'] + flt['DEP_DELAY']
# Converting date into day of week
flt['FL_DATE'] = pd.to_datetime(flt['FL_DATE'], errors='coerce')
flt['Day_of_Week'] = flt['FL_DATE'].dt.day_name()
flt_del = flt[['Day_of_Week','Total_Delay']].groupby('Day_of_Week').agg(['mean','count'])
flt_del.columns = ['mean','count']
flt_del.sort_values(by=('mean'), ascending=False)

,mean,count
Day_of_Week,,
Thursday,19.346579,282677
Friday,18.366943,286979
Monday,18.076521,259499
Wednesday,16.918030,271392
Sunday,16.340156,262897
Saturday,12.400723,228145
Tuesday,12.347663,267006


In [ ]:
fig = px.bar(flt_del, x = flt_del.index, y = 'mean', color = 'count', title = "Average Delays for Day of Week")
fig.show()

(Lighter color indicate more number of flights)
- Most flights are on Thrusdays and Fridays which also have slightly higher delay rates than the other days. This might be an indication that as the frequency of flights increase, the delays also increase. 
- However, it is quite uncanny to notice that delays on Thursdays are higher than those on Fridays! 

It was not considered to do an EDA for monthly delays since the data is provided for only a quarter which would be insufficient to make an assumption for all monthly delays.

**Delays by Carrier Operator**

In [ ]:
flt_crd = flt[['OP_CARRIER','Total_Delay']].groupby('OP_CARRIER').agg(['mean','count']).sort_values(by=('Total_Delay','mean'), ascending=False).reset_index()
flt_crd.columns = ['OP_CARRIER','mean','count']
flt_crd.head()

,OP_CARRIER,mean,count
0,C5,55.649407,11626
1,KS,51.581662,349
2,AX,38.044304,17267
3,G7,32.906797,20493
4,B6,30.890906,70847


In [ ]:
fig = px.bar(flt_crd, x=flt_crd.OP_CARRIER, y='mean', color='count', title="Delays By Carrier Airlines")
fig.show()

Lighter colors indicate more number of flights. No specific trend can be observed that shows more flights cause more delays. The carriers with highest delays are CS, KS, AX, G7.

**Tickets Dataset**

In [ ]:
tickets.shape

(706849, 12)

In [ ]:
tickets.describe()

,ITIN_ID,YEAR,QUARTER,ROUNDTRIP,PASSENGERS,ITIN_FARE
count,7.068490e+05,706849.0,706849.0,706849.0,706849.000000,706849.000000
mean,1.733162e+11,2019.0,1.0,1.0,1.936665,473.380643
std,6.681083e+10,0.0,0.0,0.0,5.041756,344.210022
min,2.019119e+06,2019.0,1.0,1.0,1.000000,0.000000
25%,2.019116e+11,2019.0,1.0,1.0,1.000000,279.000000
50%,2.019128e+11,2019.0,1.0,1.0,1.000000,416.000000
75%,2.019140e+11,2019.0,1.0,1.0,1.000000,596.000000
max,2.019153e+11,2019.0,1.0,1.0,681.000000,38400.000000


In [ ]:
tickets.median()

ITIN_ID       2.019128e+11
YEAR          2.019000e+03
QUARTER       1.000000e+00
ROUNDTRIP     1.000000e+00
PASSENGERS    1.000000e+00
ITIN_FARE     4.160000e+02
dtype: float64

The Tickets dataset has about 1.1 million datapoints. However, since this is only a sample dataset, the numbers can only be assumed to be close to exact values while analyzing. Also, apart from Passengers and ITIN_FARE, the other summary statistics for the dataset donot hold any meaningful information.  

Skewness: ITIN_FARE and Passengers column are both positively skewed since mean > median.

**Relationship between Reporting Carrier and ITIN_FARE**

In [ ]:
fig = go.Figure()

for grp in tickets.groupby(["REPORTING_CARRIER"]):
  if grp[0]!="OO":
    fig.add_trace(go.Box(y=grp[1]["ITIN_FARE"],name=grp[0]))

fig.update_layout(title="Distribution Fare by Airline Carrier",
                  xaxis_title="Ticket Reporting Carrier",
                  yaxis_title="Fare in $")
fig.show()

- From the plot above, the distribution of fares by carriers seems to have a lot of outliers, therefore, only median values shall be considered while calculating profit values using fare.
- It can also be seen that major airlines like American Airlines and United Airlines have wide range of fares, which could be because they operate in multiple routes and covers many different routes compared to other airlines.

**Airport Codes Dataset**

In [ ]:
ac.shape

(55369, 8)

Since, most variables in this dataset are non numerical variables, doing a describe would not be appropriate.

In [ ]:
ac['IATA_CODE'].dropna().nunique()

9062

In [ ]:
ac['TYPE'].value_counts(sort=True)

small_airport     34120
heliport          11384
medium_airport     4532
closed             3680
seaplane_base      1016
large_airport       614
balloonport          23
Name: TYPE, dtype: int64

There are 9062 unique origin/destination ports in the dataset, which have each been categorized into one TYPE of airports amongst small airports, medium airports, large airports, closed airports, heliports and seaplane base. The number of small airports is the maximum followed by medium and large airports. **For this project, only medium and large airports have been taken into consideration.**

**Combined Datasets EDA**

**Number of Flights Operated by each carrier throught out the quarter**

In [ ]:
nFlights_perQuarter = flights.groupby("OP_CARRIER").count()['FL_DATE'].reset_index().sort_values("FL_DATE",ascending=False).reset_index().rename({"OP_CARRIER":"CARRIER","FL_DATE":"nFlights_perQuarter"},axis=1)
nTickets_perQuarter = pd.DataFrame(tickets.groupby(["REPORTING_CARRIER"]).count()["ITIN_ID"].sort_values(ascending=False)).reset_index().rename({"REPORTING_CARRIER":"CARRIER","ITIN_ID":"nTickets_perQuarter"},axis=1)
nDistinct_routes = pd.DataFrame(tickets.groupby(["REPORTING_CARRIER","ORIGIN","DESTINATION"]).count().reset_index().groupby("REPORTING_CARRIER").count()["ORIGIN"].sort_values(ascending=False)).reset_index().rename({"REPORTING_CARRIER":"CARRIER","ORIGIN":"nDitinctRoutesCovered"},axis=1)

In [ ]:
res_plot = nFlights_perQuarter.merge(nTickets_perQuarter,on="CARRIER",how="outer").merge(nDistinct_routes,on="CARRIER",how="outer").dropna()
res_plot["nFlights_perQuarter"] = res_plot["nFlights_perQuarter"]/1000
res_plot["nTickets_perQuarter"] = res_plot["nTickets_perQuarter"]/1000
res_plot["nDitinctRoutesCovered"] = res_plot["nDitinctRoutesCovered"]/100

fig = go.Figure(data=[
    go.Bar(name='Number of Flights Per Quarter(in 1000s)', x=res_plot["CARRIER"], y=res_plot["nFlights_perQuarter"]),
    go.Bar(name='Number of Tickets per Quarter(in 1000s)', x=res_plot["CARRIER"], y=res_plot["nTickets_perQuarter"]),
    go.Bar(name='Number of Distinct Routes Covered(in 100s)', x=res_plot["CARRIER"], y=res_plot["nDitinctRoutesCovered"])
])
# Change the bar mode
fig.update_layout(barmode='group',title="Summary of Carrier in terms of Tickets Sold, Flights Completed,Distinct Routes covered in a Quarter")
fig.show()

- From this plot, it can be seen that SouthWest Airlines have the highest number of flights per quarter, but they have a very specific(concentrated) market that they run their flights in. It can also be said that not many people take that route inspite of more number of flight options(this could be because the tickets dataset only contains sample data).

- Moreover, apart fron WN, AA, DL, OO are also major players in terms of number of flights. In comparison, they have more spread out routes. 

- Smaller flights like G4 have a good ratio of number of flights and number of tickets, even when they have a lower diversity of routes (again, this analysis could be impacted if the tickets dataset would have complete data)

## **The 10 busiest round trip routes in terms of number of round trip flights in the quarter** 
(Note: Cancelled flights not included)

* Busiest route is calcuated using flights data. It provides one way flight details of different routes. Here, a round trip has been considered as count of flights from A $\rightarrow$ B and B $\rightarrow$ A. 
* Afterwards, the minimum value of those counts have been taken as the number of round trips in that route.

In [ ]:
# Since only non-cancelled flights have been considered, renaming dataframe for convenience
f_nc = flights 

# RSW-RSW not making sense hence removing it (this was found while counting number of flights in the next code chunk)
f_nc[(f_nc['ORIGIN']=="RSW") & (f_nc['DESTINATION']=="RSW")]
f_nc = f_nc.drop(f_nc[(f_nc['ORIGIN']=="RSW") & (f_nc['DESTINATION']=="RSW")].index,axis=0)

In [ ]:
# For counting of round trip flights:
# First the number of flights have been counted by grouping origin and destination for every carrier airline
# Then, the minimum count value has been taken from both way trip counts

grp_obj = f_nc.groupby(["OP_CARRIER","ORIGIN","DESTINATION"])
dfr = pd.DataFrame(grp_obj['FL_DATE'].count()).reset_index()
round_trips = pd.DataFrame([(key,dfr[((dfr['ORIGIN']==key[0]) | (dfr['ORIGIN']==key[1])) & ((dfr['DESTINATION']==key[1]) | (dfr['DESTINATION']==key[0]))]['FL_DATE'].min()) for key in grp_obj.groups.keys()])
res = round_trips.sort_values(1,ascending=False).rename({0:"roundTripRoute",1:"count"},axis=1)

dfr["roundTripRoute"] = list(zip(dfr["ORIGIN"],dfr["DESTINATION"]))
dfr = dfr[["OP_CARRIER","roundTripRoute","FL_DATE"]]
dfr=dfr[dfr['roundTripRoute'].isin(dfr.groupby('roundTripRoute').sum().sort_values("FL_DATE",ascending=False).head(20).index)]
dfr = dfr.sort_values('roundTripRoute')
dfr['roundTripRoute'] = dfr['roundTripRoute'].astype(str)
dfr = dfr.sort_values("FL_DATE",ascending=False)
dfr = dfr.rename({"FL_DATE":"Count"},axis=1)

grp_obj = f_nc.groupby(["ORIGIN","DESTINATION"])
dfr_agg = pd.DataFrame(grp_obj['FL_DATE'].count()).reset_index()
round_trips = pd.DataFrame([(key,dfr_agg[((dfr_agg['ORIGIN']==key[0]) | (dfr_agg['ORIGIN']==key[1])) & ((dfr_agg['DESTINATION']==key[1]) | (dfr_agg['DESTINATION']==key[0]))]['FL_DATE'].min()) for key in grp_obj.groups.keys()])
res = round_trips.sort_values(1,ascending=False).rename({0:"roundTripRoute",1:"count"},axis=1)
res

,roundTripRoute,count
3008,"(LAX, SFO)",4158
5277,"(SFO, LAX)",4158
4062,"(ORD, LGA)",3571
3113,"(LGA, ORD)",3571
2969,"(LAX, LAS)",3252
...,...,...
2000,"(FLG, DEN)",1
2218,"(GSP, SJC)",1
4527,"(PIE, AZA)",1
288,"(AUS, CMH)",1


In [ ]:
# Plotting Top 10 busiest round trip routes in terms of number of round trip flights in the quarter excluding canceled flights
res_plot = res.groupby("count").head(1)
fig = go.Figure([go.Bar(x=res_plot['roundTripRoute'].astype(str).head(10), 
                        y=res_plot['count'].head(10),
                        text=res_plot['count'].head(10),
                        textposition="inside")])
fig.update_layout(title="Top 10 busiest round trip routes in terms of number of round trip flights in the quarter",
                  xaxis_title="Origin-Destination Airport Codes",
                  yaxis_title="Count of Trips",
                  width=1100)
fig.show()

- The top 10 busiest round trip routes identified from this plot are:
  - LAX $\rightarrow$ SFO or SFO $\rightarrow$ LAX
  - ORD $\rightarrow$ LGA or LGA $\rightarrow$ ORD
  - LAX $\rightarrow$ LAS or LAS $\rightarrow$ LAX
  - LAX $\rightarrow$ JFK or JFK $\rightarrow$ LAX
  - SEA $\rightarrow$ LAX or LAX $\rightarrow$ SEA
  - LGA $\rightarrow$ BOS or BOS $\rightarrow$ LGA
  - OGG $\rightarrow$ HNL or HNL $\rightarrow$ OGG
  - SEA $\rightarrow$ PDX or PDX $\rightarrow$ SEA
  - ATL $\rightarrow$ MCO or MCO $\rightarrow$ ATL
  - ATL $\rightarrow$ LGA or LGA $\rightarrow$ ATL 

- Clearly, LAX-SFO has the highest number of round trip flights with a count of 4158 flights per quarter, followed by ORD-LGA which has 3571 number of round trip flights. 
- From this plot, it is also quite interesting to notice that LAX (Los Angeles) is quite a popular destination in terms of number of flights. 

However, it could be more interesting to consider OP_CARRIER in our plot to see the distribution of number of flights for each route pair.

In [ ]:
fig = px.bar(dfr, 
             x="roundTripRoute", 
             y="Count", 
             color="OP_CARRIER", 
             title="Top 20 busiest one-way trip routes by carriers")
fig.show()

- From the plot above, it can be seen that Delta Airlines(DA) have a very versatile network of routes with a good amount of flights for each route, followed by American Airlines(AA) and Alaska Airlines(AS). 

- However, higher number of flights for a route also means higher competition. For example, eventhough LAX-SFO is a very busy/popular route, it can be seen that people already have a lot of options for considering a flight for that route. 

- So, this information about only the number of round trip flights, is insufficient to determine the best possible options for choosing a lucrative route, considering the company has only 5 planes to invest on.  

## **Top 10 most Profitable Round Trips**

**Costs:**
* Fuel, Oil, Maintenance, Crew - \$8 per mile total
* Depreciation, Insurance, Other - \$1.18 per mile total
* Airport operational costs for the right to use the airports and related services are fixed at \$5,000 for medium airports and \$10,000 for large airports. There is one charge for each airport where a flight lands. Thus, a round trip flight has a total of two airport charges.
* For each individual departure, the first 15 minutes of delays are free, otherwise each minute costs the airline \$75 in added operational costs.
* For each individual arrival, the first 15 minutes of delays are free, otherwise each minute costs the airline \$75 in added operational costs.

**costs = (8 x distance) + (1.18 x distance) + (type[medium] = 5000) + (type[large] = 10000) + (delay < 15 = cost 0 and delay > 15 = 75 x delay-15)** 
<br>
<br>

**Revenue:**
* Each plane can accommodate up to 200 passengers and each flight has an
associated occupancy rate provided in the Flights data set. Do not use the
Tickets data set to determine occupancy.
* Baggage fee is \$35 for each checked bag per flight. We expect 50% of
passengers to check an average of 1 bag per flight. The fee is charged separately for each leg of a round trip flight, thus 50% of passengers will be
charged a total of \$70 in baggage fees for a round trip flight.
* Disregard seasonal effects on ticket prices (i.e. ticket prices are the same in April as they are on Memorial Day or in December)

**revenue = (occupancy_rate x 200) x ITIN_FARE + n_passengers x 0.50 x 70**

**Calculation of Profit:**

In [ ]:
# Considering only medium and large airports for analysis
ac_ml = ac[(ac["TYPE"] == 'medium_airport') | (ac["TYPE"] == 'large_airport')]
ac_ml = ac_ml[ac_ml['IATA_CODE'].notna()]

# Merging Airport_Codes data with Flights data on IATA_CODE and ORIGIN, DESTINATION respectively
f_nc_sz = f_nc.merge(ac_ml[["IATA_CODE","TYPE"]],how='left',left_on="ORIGIN",right_on="IATA_CODE").merge(ac_ml[["IATA_CODE","TYPE"]],how='left',left_on="DESTINATION",right_on="IATA_CODE")
selected_cols = ["FL_DATE","OP_CARRIER","ORIGIN","DESTINATION","DEP_DELAY","ARR_DELAY","AIR_TIME","DISTANCE","OCCUPANCY_RATE","TYPE_x","TYPE_y"]

# Airport opearional costs assigned - $5000 for medium airports and $10,000 for large airports
f_nc_sz = f_nc_sz[selected_cols].rename({"TYPE_x":"ORIGIN_size","TYPE_y":"DESTINATION_size"},axis=1)
f_nc_sz[['ORIGIN_size','DESTINATION_size']] = f_nc_sz[['ORIGIN_size','DESTINATION_size']].replace(to_replace={"large_airport":10000,"medium_airport":5000})
f_nc_sz.head()

,FL_DATE,OP_CARRIER,ORIGIN,DESTINATION,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ORIGIN_size,DESTINATION_size
0,2019-03-05,DL,ABE,ATL,-7.0,-13.0,110.0,692.0,0.52,5000.0,10000.0
1,2019-03-06,DL,ABE,ATL,-7.0,-15.0,107.0,692.0,0.99,5000.0,10000.0
2,2019-03-13,DL,ABE,ATL,-8.0,-19.0,98.0,692.0,0.72,5000.0,10000.0
3,2019-03-10,DL,ABE,ATL,-8.0,-17.0,105.0,692.0,0.40,5000.0,10000.0
4,2019-03-09,DL,ABE,ATL,-9.0,-11.0,103.0,692.0,0.89,5000.0,10000.0


In [ ]:
tickets = tickets.rename({"REPORTING_CARRIER":"OP_CARRIER"},axis=1)
# Considering only roundtrip rows from Tickets dataset
itin_fare = pd.DataFrame(tickets[tickets['ROUNDTRIP']==1].groupby(["OP_CARRIER","ORIGIN","DESTINATION"])['ITIN_FARE'].median())
temp_df = f_nc.groupby(["FL_DATE","ORIGIN","DESTINATION","TAIL_NUM"]).count().sort_values("OP_CARRIER")["OP_CARRIER"].reset_index()
temp_df=temp_df.groupby(["ORIGIN","DESTINATION"])["OP_CARRIER"].mean().reset_index()
f_nc_sz = f_nc_sz.merge(temp_df,on=["ORIGIN","DESTINATION"],how="left").rename({"OP_CARRIER_x":"OP_CARRIER","OP_CARRIER_y":"avgDailyTrip"},axis=1)
# Merging previous dataset having airport operational costs with tickets subset data on ORIGIN and DESTINATION
df_fare = pd.merge(f_nc_sz,itin_fare.reset_index(),how='left',on=["OP_CARRIER","ORIGIN","DESTINATION"])
df_fare = df_fare[df_fare["ITIN_FARE"].notna()]   #removing null of ITIN_FARE as it can canot be assumed for the flight
df_fare = df_fare.dropna()
df_fare.head()

,FL_DATE,OP_CARRIER,ORIGIN,DESTINATION,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ORIGIN_size,DESTINATION_size,avgDailyTrip,ITIN_FARE
0,2019-03-05,DL,ABE,ATL,-7.0,-13.0,110.0,692.0,0.52,5000.0,10000.0,1.00463,387.5
1,2019-03-06,DL,ABE,ATL,-7.0,-15.0,107.0,692.0,0.99,5000.0,10000.0,1.00463,387.5
2,2019-03-13,DL,ABE,ATL,-8.0,-19.0,98.0,692.0,0.72,5000.0,10000.0,1.00463,387.5
3,2019-03-10,DL,ABE,ATL,-8.0,-17.0,105.0,692.0,0.40,5000.0,10000.0,1.00463,387.5
4,2019-03-09,DL,ABE,ATL,-9.0,-11.0,103.0,692.0,0.89,5000.0,10000.0,1.00463,387.5


In [ ]:
# costs = (8*distance) + (1.18*distance) + (type[medium]*5000) + (type[large]*10000) + (delay < 15 = cost 0 and delay > 15 = 75 * delay-15)
# revenue = (occupancy_rate*200)*ITIN_FARE + n_passengers*0.50*70

# Delay costs
def delay_cost(row_val):
  if row_val>15:
    return row_val*75
  else:
    return 0

def calculate_profit(dfr):
  df = dfr.copy()
  
  df["FOMC"] = 2*8*df["DISTANCE"]    #Fuel, Oil, Maintenance, Crew - distance is twice because of round trip
  df["DIO"] = 2*1.18*df["DISTANCE"]    # Depreciation, Insurance, Other - distance is twice because of round trip
  df["AOC"] = df['ORIGIN_size']+df["DESTINATION_size"]   #Airport operational costs
  df["DELAY_COST"] = df["ARR_DELAY"].apply(lambda x: delay_cost(x)) + df["DEP_DELAY"].apply(lambda x:delay_cost(x))   # Delay Costs - considering delay values to be for round trip
  
  df["n_passengers"] = df["OCCUPANCY_RATE"]*200   # Number of passengers, considering each plane can accommodate up to 200 passengers
  df["baggage_fee"] = df["n_passengers"]*0.50*70   # Baggage fees, $70 for each bag for round trip flight
  df['fare'] = df["n_passengers"]*df["ITIN_FARE"]   # Total fare
  
  df['total_revenue'] = df["baggage_fee"]+df['fare']    # Total Revenue
  df['total_cost'] = df["FOMC"]+df["DIO"]+df["AOC"]+df["DELAY_COST"]   # Total Costs
  df["profit"] = df['total_revenue'] - df['total_cost']   # Profit = Total Revenue - Total Costs
  # Breakeven number = 90million(upfront cost for each airplane) / Profit
  return df

In [ ]:
profit_df = calculate_profit(df_fare)
profit_dfr = profit_df[profit_df["profit"]>0]
# profit_dfr = profit_dfr.groupby(["ORIGIN","DESTINATION"]).sort_values("profit",ascending=False).reset_index()
profit_dfr = profit_dfr.merge(dfr_agg,how="left",on=["ORIGIN","DESTINATION"])
profit_dfr = profit_dfr.rename({"FL_DATE_y":"busiest_roundTrips"},axis=1)
profit_dfr["breakEvenNumber"] = profit_dfr["profit"].apply(lambda x:90000000/x)
# profit_dfr['diff'] = profit_dfr['busiest_roundTrips'] - profit_dfr["breakEvenNumber"]    # calculating difference between number of round trip flights for each carrier and the breakeven number to generate a profit factor
profit_dfr = profit_dfr[profit_dfr["profit"]>0]

In [ ]:
profit_dfr.head()

,FL_DATE_x,OP_CARRIER,ORIGIN,DESTINATION,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ORIGIN_size,DESTINATION_size,avgDailyTrip,ITIN_FARE,FOMC,DIO,AOC,DELAY_COST,n_passengers,baggage_fee,fare,total_revenue,total_cost,profit,busiest_roundTrips,breakEvenNumber
0,2019-03-05,DL,ABE,ATL,-7.0,-13.0,110.0,692.0,0.52,5000.0,10000.0,1.00463,387.5,11072.0,1633.12,15000.0,0.0,104.0,3640.0,40300.0,43940.0,27705.12,16234.88,217,5543.619663
1,2019-03-06,DL,ABE,ATL,-7.0,-15.0,107.0,692.0,0.99,5000.0,10000.0,1.00463,387.5,11072.0,1633.12,15000.0,0.0,198.0,6930.0,76725.0,83655.0,27705.12,55949.88,217,1608.582539
2,2019-03-13,DL,ABE,ATL,-8.0,-19.0,98.0,692.0,0.72,5000.0,10000.0,1.00463,387.5,11072.0,1633.12,15000.0,0.0,144.0,5040.0,55800.0,60840.0,27705.12,33134.88,217,2716.170996
3,2019-03-10,DL,ABE,ATL,-8.0,-17.0,105.0,692.0,0.40,5000.0,10000.0,1.00463,387.5,11072.0,1633.12,15000.0,0.0,80.0,2800.0,31000.0,33800.0,27705.12,6094.88,217,14766.492531
4,2019-03-09,DL,ABE,ATL,-9.0,-11.0,103.0,692.0,0.89,5000.0,10000.0,1.00463,387.5,11072.0,1633.12,15000.0,0.0,178.0,6230.0,68975.0,75205.0,27705.12,47499.88,217,1894.741629


From this dataset and from the formula used for profit calculation, it can be said that the following components should be assessed further to identify the success factors:
1. Distance
2. Delays
3. Occupancy Rate
4. ITIN_Fare 
5. Busiest round trips (number of round trip flights for each unique route for each carrier) 

**Identifying and Analyzing Key Components:**

In [ ]:
key_comps = profit_dfr.sort_values("profit",ascending=False).groupby(["OP_CARRIER","ORIGIN","DESTINATION"]).head(1).reset_index()
key_comps.head()

,index,FL_DATE_x,OP_CARRIER,ORIGIN,DESTINATION,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ORIGIN_size,DESTINATION_size,avgDailyTrip,ITIN_FARE,FOMC,DIO,AOC,DELAY_COST,n_passengers,baggage_fee,fare,total_revenue,total_cost,profit,busiest_roundTrips,breakEvenNumber
0,475874,2019-03-09,AA,EGE,JFK,-3.0,0.0,207.0,1746.0,0.98,5000.0,10000.0,1.00000,2228.0,27936.0,4120.56,15000.0,0.0,196.0,6860.0,436688.0,443548.0,47056.56,396491.44,57,226.991029
1,553489,2019-03-19,UA,HNL,GUM,-11.0,-20.0,442.0,3801.0,0.99,10000.0,10000.0,1.00000,2170.0,60816.0,8970.36,20000.0,0.0,198.0,6930.0,429660.0,436590.0,89786.36,346803.64,88,259.512847
2,348918,2019-01-08,OO,DEN,DAY,-8.0,-24.0,124.0,1085.0,0.94,10000.0,10000.0,1.06962,1701.0,17360.0,2560.60,20000.0,0.0,188.0,6580.0,319788.0,326368.0,39920.60,286447.40,169,314.193810
3,676262,2019-01-02,AA,KOA,DFW,-5.0,2.0,416.0,3724.0,0.98,5000.0,10000.0,1.00000,1787.0,59584.0,8788.64,15000.0,0.0,196.0,6860.0,350252.0,357112.0,83372.64,273739.36,44,328.779902
4,552451,2019-01-07,UA,GUM,HNL,14.0,5.0,399.0,3801.0,0.99,10000.0,10000.0,1.00000,1560.0,60816.0,8970.36,20000.0,0.0,198.0,6930.0,308880.0,315810.0,89786.36,226023.64,89,398.188437


In [ ]:
key_comps[['DEP_DELAY', 'ARR_DELAY', 'AIR_TIME', 'DISTANCE', 'OCCUPANCY_RATE', 'ITIN_FARE', 'busiest_roundTrips']].describe()

,DEP_DELAY,ARR_DELAY,DISTANCE,OCCUPANCY_RATE,ITIN_FARE,busiest_roundTrips
count,9242.000000,9242.000000,9242.000000,9242.000000,9242.000000,9242.000000
mean,-2.648669,-9.851655,865.474248,0.980094,407.709370,498.215538
std,8.379009,13.849921,615.320886,0.057155,152.501239,489.475694
min,-28.000000,-71.000000,-1947.000000,0.310000,58.500000,1.000000
25%,-7.000000,-18.000000,426.250000,0.990000,310.000000,178.000000
50%,-4.000000,-10.000000,708.000000,1.000000,390.000000,348.000000
75%,-1.000000,-2.000000,1088.750000,1.000000,487.000000,648.000000
max,182.000000,188.000000,4983.000000,1.000000,2228.000000,4175.000000


In [ ]:
pd.DataFrame(key_comps[['DEP_DELAY', 'ARR_DELAY', 'AIR_TIME', 'DISTANCE', 'OCCUPANCY_RATE', 'ITIN_FARE', 'busiest_roundTrips']].median()).transpose()

,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ITIN_FARE,busiest_roundTrips
0,-4.0,-10.0,102.0,708.0,1.0,390.0,348.0


**Summary Value Analysis of Key Components:**
<br>

**Delays(Per Flight):** Since mean is slightly greater than median, so the distribution would be very slightly right skewed. Since both delays are below 15minutes, delays shall not play a major role in operational costs on a daily basis. For example, a delay of 30minutes could cost as low as $1125 which can be easily recovered just by selling ~3 tickets.

**Distance(Per Flight):** The distribution is positively skewed(right skewed) since mean > median. Along with that, it can be seen that the median distance is 708 miles which results in ~$6500 in FOMC and DI costs per flight. 

**Occupancy Rates(Per Flight):** Since median > mean, the distribution is very slightly left skewed. At 100% occupancy and 200 passeneger capacity, the airline would generate about $78000 (1 * 200 * 390) in ticket revenue. 

While in baggage revenue, they could generate $7000 (0.50 * 70 * 200).


**ITIN_FARE:** Mean>>Median, hence the distribution is right(positively) skewed. Majority of tickets are higher than $390. This could be affected by distance. 

**Busiest round trips/Day(Overall):** Mean >> Median, hence the distribution is positively skewed. This indicates that there are some routes which have higher number of round trips that would affect our decision of selecting optimum routes to maximize profits. 


<br>

After seeing the numbers, it can be said that **busiest round trips** and **ITIN_FARE** could be the most important factor, followed by **occupancy rate** and **distance** respectively. Also, from the numbers, it can be said that delays are not significant enough since most delays are below 15minutes.

Apart from these, **air_time** could also be said to be an important factor. Eventhough, it is directly proportional to distance, this can significantly reduce fuel and oil costs. This could also lead to increased number of round trips in a single day!

Based on this article (ref. link below), Boeing 757 with a passenger capacity of 200 has a turn over time of 1 hour. This means that after every trip, it takes an additional 1 hour to make the airline ready for its next flight and be made operational again.

Reference: https://www.boeing.com/commercial/aeromagazine/aero_01/textonly/t01txt.html

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Total Revenue', 
           x = [str(tup) for tup in zip(key_comps.head(10)["OP_CARRIER"],key_comps.head(10)["ORIGIN"],key_comps.head(10)["DESTINATION"])], 
           y = key_comps.head(10)["total_revenue"],
           text = key_comps.head(10)["total_revenue"],
           textposition="inside",
           insidetextanchor="start",
           texttemplate='Revenue-%{text:.4s}'),

    go.Bar(name='Total Cost', 
           x=[str(tup) for tup in zip(key_comps.head(10)["OP_CARRIER"],key_comps.head(10)["ORIGIN"],key_comps.head(10)["DESTINATION"])], 
           y=key_comps.head(10)["total_cost"]*-1,
           text=key_comps.head(10)["total_cost"]*-1,
           textposition="inside",
           texttemplate='Cost:%{text:.4s}',
           textfont=dict(
        family="sans serif",
        size=24,
        color="white"
    )),
])
fig.add_trace(
    go.Scatter(
        x=[str(tup) for tup in zip(key_comps.head(10)["OP_CARRIER"],key_comps.head(10)["ORIGIN"],key_comps.head(10)["DESTINATION"])],
        y=key_comps.head(10)["profit"],
        mode="markers+text",
        name="Gross Profit",
        text=key_comps.head(10)["profit"],
        textposition="bottom center",
        texttemplate='%{text:.4s}',
        textfont=dict(family="sans serif",
                      size=20,
                      color="black"),
        marker=dict(color='LightSkyBlue',
                    size=10,
                    line=dict(
                        color='MediumPurple',
                        width=1)
                    )
    ))
# Change the bar mode
fig.update_traces(textfont_size=15)
fig.update_layout(autosize=False,
    width=1200,
    height=800,
    barmode='stack',
    title="Profit breakdown (Revenue - Cost) of Top 10 most profitable routes",
    xaxis_title="(Carrier, Origin, Destination)",
    yaxis_title="Dollar value (in thousands)")
fig.show()

The routes shown in the graph above shows the top 10 most profitable routes in terms of the profit calculated as (revenue-cost).

- United Airlines and American Airlines surely has the most profitable routes. (They also have a very diversified fare as seen above in EDA)
- While the routes EGE $\rightarrow$ JFK, CLT $\rightarrow$ RDU, DEN $\rightarrow$ DAY or ANC $\rightarrow$ ADK has lower costs, others like HNL $\rightarrow$ GUM or KOA $\rightarrow$ DFW have higher costs. 
- Now, all factors shall be considered to decide the best profitable routes:
  - For route EGE $\rightarrow$ JFK, the profits are higher, with lower air time and distances, which reduces costs. Also, occupancy rate is higher which increases revenue. 
  - Therefore, considering only profit values, the most important factors can be said to be:
    - Fare
    - Distance
    - Air time
    - Occupancy rate 

## **The 5 round trip routes that I would recommend to invest in based on the factors chosen above**

#### **Case 1**

Assumption: 

Number of Roundtrips per day = Total Round Trip routes/Number of Days in a Quarter(90days)

In [ ]:
key_comps["RoundTrips_perday"] = key_comps["busiest_roundTrips"]/90
top5_case1 = key_comps.sort_values(["profit","RoundTrips_perday"],ascending=(False, False)).head(5)[["OP_CARRIER", "ORIGIN", "DESTINATION","ITIN_FARE","total_revenue","total_cost","RoundTrips_perday","profit","busiest_roundTrips", "breakEvenNumber","avgDailyTrip"]]
top5_case1["Suggestedroundtrips"] = [2,1,3,1,1]   # considering air_time: air_time around 2hrs => 3 trips and air_time about 3-4 or more hours => 2 trips, air_time > 4 hours => 1 trip
top5_case1["DaysToBreakEven"]=top5_case1["breakEvenNumber"]/top5_case1["Suggestedroundtrips"]
top5_case1

,OP_CARRIER,ORIGIN,DESTINATION,ITIN_FARE,total_revenue,total_cost,RoundTrips_perday,profit,busiest_roundTrips,breakEvenNumber,avgDailyTrip,Suggestedroundtrips,DaysToBreakEven
0,AA,EGE,JFK,2228.0,443548.0,47056.56,0.633333,396491.44,57,226.991029,1.00000,2,113.495515
1,UA,HNL,GUM,2170.0,436590.0,89786.36,0.977778,346803.64,88,259.512847,1.00000,1,259.512847
2,OO,DEN,DAY,1701.0,326368.0,39920.60,1.877778,286447.40,169,314.193810,1.06962,3,104.731270
3,AA,KOA,DFW,1787.0,357112.0,83372.64,0.488889,273739.36,44,328.779902,1.00000,1,328.779902
4,UA,GUM,HNL,1560.0,315810.0,89786.36,0.988889,226023.64,89,398.188437,1.00000,1,398.188437


In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Total Revenue', 
           x = [str(tup) for tup in zip(top5_case1["OP_CARRIER"],top5_case1["ORIGIN"],top5_case1["DESTINATION"])], 
           y = top5_case1["total_revenue"],
           text = top5_case1["total_revenue"],
           textposition="inside",
           insidetextanchor="start",
           texttemplate='Revenue-%{text:.4s}'),

    go.Bar(name='Total Cost', 
           x=[str(tup) for tup in zip(top5_case1["OP_CARRIER"],top5_case1["ORIGIN"],top5_case1["DESTINATION"])], 
           y=top5_case1["total_cost"]*-1,
           text=top5_case1["total_cost"]*-1,
           textposition="inside",
           texttemplate='Cost:%{text:.4s}',
           textfont=dict(
        family="sans serif",
        size=24,
        color="white"
    )),
])
fig.add_trace(
    go.Scatter(
        x=[str(tup) for tup in zip(top5_case1["OP_CARRIER"],top5_case1["ORIGIN"],top5_case1["DESTINATION"])],
        y=top5_case1["profit"],
        mode="markers+text",
        name="Gross Profit",
        text=top5_case1["profit"],
        textposition="bottom center",
        texttemplate='%{text:.4s}',
        textfont=dict(family="sans serif",
                      size=20,
                      color="black"),
        marker=dict(color='LightSkyBlue',
                    size=10,
                    line=dict(
                        color='MediumPurple',
                        width=1)
                    )
    ))
# Change the bar mode
fig.update_traces(textfont_size=15)
fig.update_layout(autosize=False,
    width=1000,
    height=800,
    barmode='stack',
    title="Profit breakdown (Revenue - Cost) of Top 5 most profitable routes",
    xaxis_title="(Carrier, Origin, Destination)",
    yaxis_title="Dollar value (in thousands)")
fig.show()

From the plot above, the most profitable routes are :
- EGE $\rightarrow$ JFK
- HNL $\rightarrow$ GUM
- DEN $\rightarrow$ DAY
- KOA $\rightarrow$ DFW
- GUM $\rightarrow$ HNL

Insights:<br>
- Focus on lower number of round trips per day, as limited number of flights with higher occupancy rates from these location pairs will drive higher profit margins.
- These routes have the highest profits mostly because they have higher fares. The high fare could be because of greater distances and air_time. So, if the airline company wishes to charge higher fares for these routes, they can possibly expect to achieve higher profits. 

Key factors focussed on:<br>
- Higher Margin per Round Trip
- Lower Number of trips per day to generate higher margin. (Less operational cost per route)


In this case however, there could be other risks of choosing these routes, which shall be discussed further.

#### **Case 2**

Assumptions:
* Tail number and Flight Date is given in Flights Dataset.
* Calculate Average RoundTrip done by a single plane on the given route per Day(identified based on Tail Number).
* Calculated avgDailyTrips can help make assumption about how many trips a single plane is doing on the given route.

In [ ]:
key_comps['possible_trips/Quarter'] = key_comps["avgDailyTrip"]*90
key_comps["numberQuarterToBreakeven"] = key_comps["breakEvenNumber"]/key_comps["possible_trips/Quarter"]
key_comps = key_comps[key_comps["busiest_roundTrips"]>key_comps["possible_trips/Quarter"]]
top5_case2 = key_comps.sort_values("numberQuarterToBreakeven").head(5)
top5_case2[["OP_CARRIER","ORIGIN","DESTINATION","avgDailyTrip","total_revenue","total_cost","profit","busiest_roundTrips","breakEvenNumber","possible_trips/Quarter","numberQuarterToBreakeven"]]

,OP_CARRIER,ORIGIN,DESTINATION,avgDailyTrip,total_revenue,total_cost,profit,busiest_roundTrips,breakEvenNumber,possible_trips/Quarter,numberQuarterToBreakeven
2,OO,DEN,DAY,1.069620,326368.0,39920.60,286447.40,169,314.193810,96.265823,3.263815
622,AA,DFW,BDL,2.788079,143550.0,47007.56,96542.44,421,932.232498,250.927152,3.715152
96,9E,MSP,CWA,1.948454,153900.0,18213.00,135687.00,189,663.291251,175.360825,3.782437
6,OH,CLT,RDU,1.072902,236412.0,22386.80,214025.20,780,420.511230,96.561210,4.354867
21,9E,EYW,ATL,1.231362,206910.0,26860.56,180049.44,479,499.862704,110.822622,4.510475


In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Total Revenue', 
           x = [str(tup) for tup in zip(top5_case2["OP_CARRIER"],top5_case2["ORIGIN"],top5_case2["DESTINATION"])], 
           y = top5_case2["total_revenue"],
           text = top5_case2["total_revenue"],
           textposition="inside",
           insidetextanchor="start",
           texttemplate='Revenue-%{text:.4s}'),

    go.Bar(name='Total Cost', 
           x=[str(tup) for tup in zip(top5_case2["OP_CARRIER"],top5_case2["ORIGIN"],top5_case2["DESTINATION"])], 
           y=top5_case2["total_cost"]*-1,
           text=top5_case2["total_cost"]*-1,
           textposition="inside",
           texttemplate='Cost:%{text:.4s}',
           textfont=dict(
        family="sans serif",
        size=24,
        color="white"
    )),
])
fig.add_trace(
    go.Scatter(
        x=[str(tup) for tup in zip(top5_case2["OP_CARRIER"],top5_case2["ORIGIN"],top5_case2["DESTINATION"])],
        y=top5_case2["profit"],
        mode="markers+text",
        name="Gross Profit",
        text=top5_case2["profit"],
        textposition="bottom center",
        texttemplate='%{text:.4s}',
        textfont=dict(family="sans serif",
                      size=20,
                      color="black"),
        marker=dict(color='LightSkyBlue',
                    size=10,
                    line=dict(
                        color='MediumPurple',
                        width=1)
                    )
    ))
# Change the bar mode
fig.update_traces(textfont_size=15)
fig.update_layout(autosize=False,
    width=1000,
    height=800,
    barmode='stack',
    title="Profit breakdown (Revenue - Cost) of Top 5 most profitable routes",
    xaxis_title="(Carrier, Origin, Destination)",
    yaxis_title="Dollar value (in thousands)")
fig.show()

From the plot above, the most profitable routes are :
- DEN $\rightarrow$ DAY
- DFW $\rightarrow$ BDL
- MSP $\rightarrow$ CWA
- CLT $\rightarrow$ RDU
- EYW $\rightarrow$ ATL

Insights:<br>
- Asset utilization is an important factor for a smaller carrier, the company can work on running more flights every day on lower margin routes and less busy routes.
- As demand on the route can vary highly, it could be risky in Case 1 to lose demand and huge revenue can be lost by even missing a single flight.

Key factors focussed on:<br>
- Higher Margin with more number of flights


## **The number of round trip flights it will take to breakeven on the upfront airplane cost for each of the 5 round trip routes**

**According to my suggestions, the breakeven calculation is as follows:**<br>
The BreakEvenNumber calculated shows the number of flights a carrier needs to make in order to reach to the breakeven cost.

#### **Case 1**

In [ ]:
top5_case1[["ORIGIN","DESTINATION","breakEvenNumber","busiest_roundTrips"]].head()

,ORIGIN,DESTINATION,breakEvenNumber,busiest_roundTrips
0,EGE,JFK,226.991029,57
1,HNL,GUM,259.512847,88
2,DEN,DAY,314.193810,169
3,KOA,DFW,328.779902,44
4,GUM,HNL,398.188437,89


#### **Case 2**

In [ ]:
top5_case2[["ORIGIN","DESTINATION","breakEvenNumber","busiest_roundTrips","possible_trips/Quarter","numberQuarterToBreakeven"]].head()

,ORIGIN,DESTINATION,breakEvenNumber,busiest_roundTrips,possible_trips/Quarter,numberQuarterToBreakeven
2,DEN,DAY,314.193810,169,96.265823,3.263815
622,DFW,BDL,932.232498,421,250.927152,3.715152
96,MSP,CWA,663.291251,189,175.360825,3.782437
6,CLT,RDU,420.511230,780,96.561210,4.354867
21,EYW,ATL,499.862704,479,110.822622,4.510475


Considering both cases, it would require more number of flights (breakEvenNumber) to reach the breakeven point when the margins are lower for each trip (case 2). In that case, with the possible trips per quarter rate, it would require about 3-4 quarters to reach the breakeven point. 

## **Key Performance Indicators (KPI’s) that I would recommend tracking in the future to measure the success of the round trip routes**

- **Number of round trip flights:** As seen in Case 2 analysis, the company can yeild higher profits if they run more number of flights per day for the profitable routes. The company can use the higher frequency of flights along with shorter distances to rack up more revenue and get themselves ahead in the game.  

- **Occupancy Rate:** If occupancy rate is high, the airline could bring in more ticket revenue as well as baggage revenue. Therefore, it is important to track the occupancy rate for every round trip.

- **Distance:** Distance is a key factor that should be taken into account. Coming into this new industry the airline might want to focus on the samller routes in the beginning. The idea behind selecting shorter routes is that the fare is not proportional to distance and they can generate significant revenue using that.

## **Conclusion**

The airline business can be very lucrative if key factors are identified and focussed on. Since the company is very new in the business, they definitely need to see how their competitors are operating ad performing in different markets. That is what this analysis has focussed on. 

**Overall Analysis and my recommendations:**

- Considering all the key components such as distance, fare, occupancy, profit per flight and number of roundtrips per day, also taking into account the initial investment of $90 million for 5 planes only, it can be said that the company can generate good revenue if they focus on Origin-Destination pairs having higher foot traffic and higher number of round trips per day per carrier! In that case, I would suggest considering round trips for the routes in Case 2. 

- More number of flights can really benefit from high occupancy rates from these location pairs as well as low maintenance costs for covering shorter distances, which would drive the profit margins higher!

- There are smaller key factors such as air time that can help us determine how many round trips the airline can make per day to maximize their profits.

## **Future Scopes**

- The accuracy of the analysis can be improved if the actual number of ticket data can be provided
- A more generalized calculation of profits can be done, not considering groupwise calculations for airline carriers, which would give us better numbers for the overall market routes
- If actual profit values can be attained, statistical analysis like feature importance, hypothesis testing etc. can be performed to understand KPIs better 

## **Metadata Created**

- **roundTripRoute**: Flight routes from A to B and B to A
- **FOMC**: Fuel, Oil, Maintenance, Crew costs calculated using 2*8*distance
- **DIO**: Depreciation, Insurance, Other Costs calculated using 2*1.18*distance
- **AOC**: Airport operational costs calculated by Origin and Destination landing drop off costs for medium($5000) and large($10000) airport respectively
- **DELAY_COST**: Arrival+Departure Delay Costs
- **n_passengers**: Number of passengers occupying the flight calculated using OCCUPANCY_RATEx200 (200 seating capacity of boeing)
- **baggage_fee**: n_passengers*0.50*70 (50% of passengers having extra baggage fee cost of $70 for a roundtrip)
- **fare**: Total fare for each round trip flight calculated using n_passengersxITIN_FARE
- **total_revenue**: Includes baggage fees and fare for each round trip
- **total_cost**: Includes FOMC, DIO, AOC and delay costs for each round trip
- **profit**: Subtracted value of total_revenue and total_costs
- **busiest_roundTrips**: Number of round trip flights from A to B and B to A combined (lower value of each way count)
- **avgDailyTrip**: Total round trips for one single flight, calculated for each flight using tail_number.  
- **breakEvenNumber**: Number of round trips required to cover the initial investement of 90 million
- **RoundTrips_perday**: Total round trips per day(i.e. the busiest_roundTrips divided by 90)
- **Suggestedroundtrips**: Suggested values of round trips based on air_time. For air_time around 2hours, 3 trips are suggested, for air_time about 3-4hours or more, 2 trips are suggested, and for air_time greater than 4hours, 1 trip is suggested.
- **DaysToBreakEven**: Number of days required for each round trip route flight to reach the breakeven point 
- **possible_trips/Quarter**: Number of flights possible to run in a quiarter, calcuated by using average daily trips multiplied by 90 for the quarter.
- **numberQuarterToBreakeven**: Quarters required to reach breakeven number